In [ ]:
"""
Simulation of hydrogen orbitals with a graphical interface
Allows selecting the quantum numbers; I used the Monte Carlo method.
The scene can be rotated in real time with the mouse.
A yellow nucleus is added to distinguish the central region
"""

from vpython import canvas, sphere, vector, color, slider, wtext, button, rate, scene
import random, math

current_n = 2
current_l = 1
current_m = 0

range_r = 10.0      # Maximum radius for sampling
num_points = 10000  # Number of Monte Carlo sample points

sample_points = []  
spheres_list = []   
rotation_angle = 0  

# Radial wavefunction for hydrogen-like orbitals
def radial(n, l, r, a0=1.0):
    if n == 1 and l == 0:       # 1s
        return 2.0 * math.exp(-r)
    elif n == 2 and l == 0:     # 2s
        return (1.0/(2.0*math.sqrt(2))) * (2 - r) * math.exp(-r/2)
    elif n == 2 and l == 1:     # 2p
        return (1.0/(2.0*math.sqrt(6))) * r * math.exp(-r/2)
    elif n == 3 and l == 0:     # 3s (approximate)
        return (1.0/math.sqrt(27)) * (27 - 18*r + 2*r**2) * math.exp(-r/3)
    elif n == 3 and l == 1:     # 3p (approximate)
        return (1.0/math.sqrt(27)) * r * (6 - r) * math.exp(-r/3)
    elif n == 3 and l == 2:     # 3d (approximate)
        return (1.0/math.sqrt(27)) * (r**2) * math.exp(-r/3)
    elif n == 4 and l == 0:     # 4s (approximate)
        return (1.0/math.sqrt(64)) * (1 - r/4 + r**2/32 - r**3/384) * math.exp(-r/4)
    return 0.0

# Spherical harmonics up to l=3
def spherical_harmonic(l, m, theta, phi):
    if l == 0 and m == 0:
        return 1.0 / math.sqrt(4*math.pi)
    elif l == 1:
        if m == 0:
            return math.sqrt(3/(4*math.pi)) * math.cos(theta)
        elif m == 1:
            return math.sqrt(3/(4*math.pi)) * math.sin(theta) * math.cos(phi)
        elif m == -1:
            return math.sqrt(3/(4*math.pi)) * math.sin(theta) * math.sin(phi)
    elif l == 2:
        if m == 0:
            return math.sqrt(5/(16*math.pi)) * (3*math.cos(theta)**2 - 1)
        elif m == 1:
            return math.sqrt(15/(4*math.pi)) * math.sin(theta)*math.cos(theta)*math.cos(phi)
        elif m == -1:
            return math.sqrt(15/(4*math.pi)) * math.sin(theta)*math.cos(theta)*math.sin(phi)
        elif m == 2:
            return math.sqrt(15/(16*math.pi)) * (math.sin(theta)**2) * math.cos(2*phi)
        elif m == -2:
            return math.sqrt(15/(16*math.pi)) * (math.sin(theta)**2) * math.sin(2*phi)
    elif l == 3:
        if m == 0:
            return math.sqrt(7/(16*math.pi)) * (5*math.cos(theta)**3 - 3*math.cos(theta))
        elif m == 1:
            return math.sqrt(21/(64*math.pi)) * math.sin(theta)*(5*math.cos(theta)**2 - 1)*math.cos(phi)
        elif m == -1:
            return math.sqrt(21/(64*math.pi)) * math.sin(theta)*(5*math.cos(theta)**2 - 1)*math.sin(phi)
        elif m == 2:
            return math.sqrt(105/(32*math.pi)) * (math.sin(theta)**2)*math.cos(theta)*math.cos(2*phi)
        elif m == -2:
            return math.sqrt(105/(32*math.pi)) * (math.sin(theta)**2)*math.cos(theta)*math.sin(2*phi)
        elif m == 3:
            return math.sqrt(35/(64*math.pi)) * (math.sin(theta)**3)*math.cos(3*phi)
        elif m == -3:
            return math.sqrt(35/(64*math.pi)) * (math.sin(theta)**3)*math.sin(3*phi)
    return 0.0

# Full wavefunction ψ = R(r)·Y(θ,φ)
def psi(n, l, m, r, theta, phi):
    return radial(n, l, r) * spherical_harmonic(l, m, theta, phi)

# Monte Carlo sampling of orbital points
def generate_orbital_points(n, l, m, range_r, num_points):
    # Estimate max probability density for rejection sampling
    p_max = 0
    samples_for_max = 10000
    for _ in range(samples_for_max):
        r_test = range_r * (random.random() ** (1/3))
        theta_test = math.acos(1 - 2*random.random())
        phi_test = 2*math.pi*random.random()
        p = (psi(n, l, m, r_test, theta_test, phi_test)**2) * (r_test**2)
        if p > p_max:
            p_max = p

    pts = []
    sph_list = []
    for _ in range(num_points):
        # Random spherical coordinates
        r = range_r * (random.random() ** (1/3))
        theta = math.acos(1 - 2*random.random())
        phi = 2*math.pi*random.random()
        psi_val = psi(n, l, m, r, theta, phi)
        p = (psi_val**2) * (r**2)
        # Accept or reject based on p_max
        if random.random() * p_max < p:
            pos = vector(r * math.sin(theta) * math.cos(phi),
                         r * math.sin(theta) * math.sin(phi),
                         r * math.cos(theta))
            pts.append((r, theta, phi, pos))
            dens = psi_val**2
            col_val = min(dens / p_max, 1)
            # Color mapping: blue for low density, white for high
            s = sphere(pos=pos, radius=0.1, color=vector(col_val, col_val, 1-col_val), opacity=0.8)
            sph_list.append(s)
    return pts, sph_list

# Rotate a vector around the y-axis by a given angle
def rotate_y(v, angle):
    cos_a = math.cos(angle)
    sin_a = math.sin(angle)
    return vector(v.x*cos_a + v.z*sin_a, v.y, -v.x*sin_a + v.z*cos_a)

# Create UI sliders for quantum numbers
def create_sliders():
    global n_slider, l_slider, m_slider, n_text, l_text, m_text, update_button
    scene.caption = ""  # Reset caption area
    scene.append_to_caption("\nSelect quantum numbers:\n\n")
    n_slider = slider(min=1, max=4, value=current_n, length=200, bind=update_n, right=15, step=1)
    n_text = wtext(text=" n = " + str(current_n) + "\n\n")
    l_slider = slider(min=0, max=current_n-1, value=current_l, length=200, bind=update_l, right=15, step=1)
    l_text = wtext(text=" l = " + str(current_l) + "\n\n")
    m_slider = slider(min=-current_l, max=current_l, value=current_m, length=200, bind=update_m, right=15, step=1)
    m_text = wtext(text=" m = " + str(current_m) + "\n\n")
    update_button = button(text="Update Orbital", bind=update_orbital)
    scene.append_to_caption("\nDrag the mouse on the scene to rotate it.\n")

# Callback functions to update quantum numbers
def update_n(s):
    global current_n, current_l, current_m
    current_n = int(s.value)
    if current_l >= current_n:
        current_l = current_n - 1
    create_sliders()

def update_l(s):
    global current_l, current_m
    current_l = int(s.value)
    if abs(current_m) > current_l:
        current_m = 0
    create_sliders()

def update_m(s):
    global current_m, m_text
    current_m = int(s.value)
    m_text.text = " m = " + str(current_m) + "\n\n"

# Regenerate the orbital when button is pressed
def update_orbital(b):
    global sample_points, spheres_list
    for s in spheres_list:
        s.visible = False
    spheres_list.clear()
    sample_points.clear()
    sample_points, new_sph = generate_orbital_points(current_n, current_l, current_m, range_r, num_points)
    spheres_list.extend(new_sph)

# Scene setup
scene.width = 800
scene.height = 600
scene.background = color.black
scene.title = "Interactive Hydrogen Orbital Visualization\n"
create_sliders()

# Mouse movement to control rotation
def mouse_move(evt):
    global rotation_angle
    rotation_angle = evt.pos.x * 0.01

scene.bind('mousemove', mouse_move)

# Draw the nucleus at the origin
nucleus = sphere(pos=vector(0, 0, 0), radius=0.1, color=color.orange, opacity=1)

# Main loop: rotate points in real time
while True:
    rate(60)
    for i, (r, theta, phi, orig_pos) in enumerate(sample_points):
        new_pos = rotate_y(orig_pos, rotation_angle)
        spheres_list[i].pos = new_pos

<IPython.core.display.Javascript object>